In [7]:
import cv2
from IPython.display import clear_output
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from utils import ones_state, vis, state_ones

cam = cv2.VideoCapture(1)

In [8]:
from detector import Detector
d = Detector(64, 64, "weights-00-0.28.hdf5")

c:\Users\aabdo\miniconda3\envs\tic_tac_toe\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


model created with # params: 507858
loading model weights from path: weights-00-0.28.hdf5
loaded weights


In [3]:
# create tictac ai
import t3_ai
t3 = t3_ai.AI(weights_path="t3_weights-19-0.24.hdf5")

model created with # params: 353
loading model weights from path: t3_weights-19-0.24.hdf5
loaded weights


c:\Users\aabdo\miniconda3\envs\tic_tac_toe\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
def rc_from_int(a):
    r = int(np.floor(a/3.0))
    c = a%3
    return r,c

In [6]:
import cv2
# from IPython.display import clear_output

# now let's initialize the list of reference point
ref_point = []
draw = False
def shape_selection(event, x, y, flags, param):
    # grab references to the global variables
    global ref_point, crop, draw
    # ix, iy = (0, 0)
    # if the left mouse button was clicked, record the starting
    # (x, y) coordinates and indicate that cropping is being performed
    if event == cv2.EVENT_LBUTTONDOWN:
        draw = True
        ref_point = [(x, y)]
        # image = clone.copy()
        # cv2.imshow("image", image)

    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates and indicate that
        # the cropping operation is finished
        if draw:
            draw = False
            ref_point.append((x, y))
            ix, iy = (ref_point[0][0], ref_point[0][1])
            # calculate the side length of the square
            side_length = max(abs(x - ix), abs(y - iy))

            # determine the bottom-right corner of the square
            if x >= ix and y >= iy:
                ref_point[1] = (ix + side_length, iy + side_length)
            elif x >= ix and y < iy:
                ref_point[1] = (ix + side_length, iy - side_length)
            elif x < ix and y >= iy:
                ref_point[1] = (ix - side_length, iy + side_length)
            else:
                ref_point[1] = (ix - side_length, iy - side_length)

                        
            # cv2.imshow("image", image)




# load the image, clone it, and setup the mouse callback function
ret_val, image = cam.read()
# image = cv2.imread('download.png')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", shape_selection)

croped_img = None
# keep looping until the 'q' key is pressed
rotation = 0
while True:
    # display the image and wait for a keypress
    ret_val, image = cam.read()
    for _ in range(rotation):
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    
    if len(ref_point)==2:
        # draw a square around the region of interest
        thickness = 2
        cv2.rectangle(image, 
                (ref_point[0][0] - thickness, ref_point[0][1] - thickness), 
                (ref_point[1][0] + thickness, ref_point[1][1] + thickness), 
                (0, 255, 0), thickness)
        croped_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        croped_img = croped_img[ref_point[0][1]:ref_point[1][1], ref_point[0][0]:ref_point[1][0]]
        croped_img = cv2.resize(croped_img, (64, 64))
        croped_img = cv2.adaptiveThreshold(croped_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11,2)
        croped_img = 255-croped_img
        croped_img = croped_img.astype(np.float64)
        croped_img /= 255

        clear_output(wait=True)
        cv2.imshow("cropped", cv2.resize(croped_img, (256, 256)))
        # plt.imshow(croped_img, cmap=plt.cm.gray)
        state = np.round(d.predict(croped_img))
        vis(ones_state(state))
        state = ones_state(state)
        # besta = t3.best_move(state)
        # r, c = rc_from_int(besta)
        # print(r,c)
    
    cv2.imshow("image", image)
    
    key = cv2.waitKey(1) & 0xFF

    # press 'r' to reset the window
    if key == ord("r"):
        # draw = True
        rotation += 1
        rotation %= 4
        # image = clone.copy()

    # if the 'c' key is pressed, break from the loop
    elif key == 27:
        break

# close all open windows
print(ref_point)
cv2.destroyAllWindows() 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

O O X 
X X - 
O - X 
[(230, 251), (432, 453)]
